# Assignment 2 - Database Design and Data Aggregation

Student Name: Nirav Patel

Student ID: 40248940

Date: March 21, 2025

> The Python scripts that you will see in this file, and that can be viewed in the `scripts` folder use environment variables.

> You may set the required variables as explained in the `README.md` if you wish to run the scripts.


## Question 1: Starwars API

We begin by providing the DDL that is used to create the tables for the first data source, which is the Starwars API.

This file is conatined in the `ddl` folder. Running this SQL code is NOT required as the following Python code handles it.


In [ ]:
-- Drop tables if they exist
DROP TABLE IF EXISTS people_films CASCADE;
DROP TABLE IF EXISTS people_species CASCADE;
DROP TABLE IF EXISTS people_starships CASCADE;
DROP TABLE IF EXISTS people_vehicles CASCADE;
DROP TABLE IF EXISTS films_species CASCADE;
DROP TABLE IF EXISTS films_starships CASCADE;
DROP TABLE IF EXISTS films_vehicles CASCADE;
DROP TABLE IF EXISTS films_planets CASCADE;

DROP TABLE IF EXISTS people CASCADE;
DROP TABLE IF EXISTS films CASCADE;
DROP TABLE IF EXISTS starships CASCADE;
DROP TABLE IF EXISTS vehicles CASCADE;
DROP TABLE IF EXISTS species CASCADE;
DROP TABLE IF EXISTS planets CASCADE;

-- Create tables
CREATE TABLE people (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    birth_year VARCHAR(20),
    eye_color VARCHAR(50),
    gender VARCHAR(20),
    hair_color VARCHAR(50),
    height VARCHAR(20),
    mass VARCHAR(20),
    skin_color VARCHAR(50),
    homeworld VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE films (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    episode_id INTEGER NOT NULL,
    opening_crawl TEXT,
    director VARCHAR(255),
    producer VARCHAR(255),
    release_date DATE,
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE starships (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    model VARCHAR(255),
    starship_class VARCHAR(255),
    manufacturer VARCHAR(255),
    cost_in_credits VARCHAR(50),
    length VARCHAR(50),
    crew VARCHAR(50),
    passengers VARCHAR(50),
    max_atmosphering_speed VARCHAR(50),
    hyperdrive_rating VARCHAR(50),
    MGLT VARCHAR(50),
    cargo_capacity VARCHAR(50),
    consumables VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE vehicles (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    model VARCHAR(255),
    vehicle_class VARCHAR(255),
    manufacturer VARCHAR(255),
    length VARCHAR(50),
    cost_in_credits VARCHAR(50),
    crew VARCHAR(50),
    passengers VARCHAR(50),
    max_atmosphering_speed VARCHAR(50),
    cargo_capacity VARCHAR(50),
    consumables VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE species (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    classification VARCHAR(255),
    designation VARCHAR(50),
    average_height VARCHAR(50),
    average_lifespan VARCHAR(50),
    eye_colors VARCHAR(255),
    hair_colors VARCHAR(255),
    skin_colors VARCHAR(255),
    language VARCHAR(255),
    homeworld VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE planets (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    diameter VARCHAR(50),
    rotation_period VARCHAR(50),
    orbital_period VARCHAR(50),
    gravity VARCHAR(50),
    population VARCHAR(50),
    climate VARCHAR(255),
    terrain VARCHAR(255),
    surface_water VARCHAR(50),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

-- Relationship Tables for Many-to-Many Relationships
CREATE TABLE people_films (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, film_url)
);

CREATE TABLE people_species (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    species_url VARCHAR(255) REFERENCES species(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, species_url)
);

CREATE TABLE people_starships (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    starship_url VARCHAR(255) REFERENCES starships(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, starship_url)
);

CREATE TABLE people_vehicles (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    vehicle_url VARCHAR(255) REFERENCES vehicles(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, vehicle_url)
);

CREATE TABLE films_species (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    species_url VARCHAR(255) REFERENCES species(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, species_url)
);

CREATE TABLE films_starships (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    starship_url VARCHAR(255) REFERENCES starships(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, starship_url)
);

CREATE TABLE films_vehicles (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    vehicle_url VARCHAR(255) REFERENCES vehicles(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, vehicle_url)
);

CREATE TABLE films_planets (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    planet_url VARCHAR(255) REFERENCES planets(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, planet_url)
);

## Question 2

We now provide the code used to fetch data for the desired entities from the API using Python.


In [33]:
import requests
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Access the environment variables
DB_PARAMS = {
    "dbname": os.getenv("UNNORMALIZED_DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT")
}

BASE_URL = "https://swapi.dev/api/"
SCHEMA_FILE = "ddl/01_starwars_api_tables.sql"

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(**DB_PARAMS)
    cursor = conn.cursor()

    # Read the SQL schema file
    with open(SCHEMA_FILE, "r", encoding="utf-8") as file:
        sql_script = file.read()

    # Execute the SQL script
    cursor.execute(sql_script)
    conn.commit()

    print("Star Wars Schema executed successfully.")

except Exception as e:
    print(f"Error executing schema: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()

# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(**DB_PARAMS)

# Fetch data from SWAPI with pagination
def fetch_data(endpoint):
    url = BASE_URL + endpoint
    data = []
    while url:
        response = requests.get(url)
        if response.status_code == 200:
            json_data = response.json()
            data.extend(json_data["results"])
            url = json_data["next"]  # Pagination
        else:
            print(f"Failed to fetch {endpoint}: {response.status_code}")
            break
    return data

# Insert data into database
def insert_data(table, data, columns):
    conn = get_db_connection()
    cur = conn.cursor()

    # Exclude relationship columns from the insert data
    non_relationship_columns = [col for col in columns if col not in ["films", "species", "starships", "vehicles", "planets"]]

    query = f"""
        INSERT INTO {table} ({', '.join(non_relationship_columns)})
        VALUES %s
        ON CONFLICT DO NOTHING
    """
    values = []
    for record in data:
        row = []
        for col in non_relationship_columns:
            if isinstance(record.get(col), list):
                row.append("{" + ",".join(record.get(col, [])) + "}")  # Convert list to PostgreSQL array format
            else:
                row.append(record.get(col, None))
        values.append(row)

    try:
        print("Inserting non-relationship data into table:", table)
        execute_values(cur, query, values)
        conn.commit()

    except Exception as e:
        print(f"Error inserting non-relationship data into {table}: {e}")
        conn.rollback()  # Rollback on error
    finally:
        cur.close()

    return conn, table, data, columns

def insert_relationship_data(conn, table, data, columns):
    cur = conn.cursor()

    # Phase 2: Insert relationship data for all tables
    if "films" in columns:
        for record in data:
            for film_url in record.get("films", []):
                print(f"Inserting into people_films: person_url={record['url']} film_url={film_url}")
                execute_values(cur, f"""
                    INSERT INTO people_films (person_url, film_url)
                    VALUES %s
                """, [(record["url"], film_url)])

    # Insert relationships for "species"
    if "species" in columns:
        for record in data:
            for species_url in record.get("species", []):
                if table == "people":  # Inserting into people_starships
                    print(f"Inserting into people_species: person_url={record['url']} species_url={species_url}")
                    execute_values(cur, f"""
                        INSERT INTO people_species (person_url, species_url)
                        VALUES %s
                    """, [(record["url"], species_url)])
                elif table == "films":  # Inserting into films_starships
                    print(f"Inserting into films_species: film_url={record['url']} species_url={species_url}")
                    execute_values(cur, f"""
                        INSERT INTO films_species (film_url, species_url)
                        VALUES %s
                    """, [(record["url"], species_url)])

    # Insert relationships for "starships"
    if "starships" in columns:
        for record in data:
            for starship_url in record.get("starships", []):
                if table == "people":  # Inserting into people_starships
                    print(f"Inserting into people_starships: person_url={record['url']} starship_url={starship_url}")
                    execute_values(cur, f"""
                        INSERT INTO people_starships (person_url, starship_url)
                        VALUES %s
                    """, [(record["url"], starship_url)])
                elif table == "films":  # Inserting into films_starships
                    print(f"Inserting into films_starships: film_url={record['url']} starship_url={starship_url}")
                    execute_values(cur, f"""
                        INSERT INTO films_starships (film_url, starship_url)
                        VALUES %s
                    """, [(record["url"], starship_url)])

    # Insert relationships for "vehicles"
    if "vehicles" in columns:
        for record in data:
            for vehicle_url in record.get("vehicles", []):
                if table == "people":  # Inserting into people_vehicles
                    print(f"Inserting into people_vehicles: person_url={record['url']} vehicle_url={vehicle_url}")
                    execute_values(cur, f"""
                        INSERT INTO people_vehicles (person_url, vehicle_url)
                        VALUES %s
                    """, [(record["url"], vehicle_url)])
                elif table == "films":  # Inserting into films_vehicles
                    print(f"Inserting into films_vehicles: film_url={record['url']} vehicle_url={vehicle_url}")
                    execute_values(cur, f"""
                        INSERT INTO films_vehicles (film_url, vehicle_url)
                        VALUES %s
                    """, [(record["url"], vehicle_url)])

    # Insert relationships for "planets"
    if "planets" in columns:
        for record in data:
            for planet_url in record.get("planets", []):
                print(f"Inserting into films_planets: film_url={record['url']} planet_url={planet_url}")
                execute_values(cur, f"""
                    INSERT INTO films_planets (film_url, planet_url)
                    VALUES %s
                """, [(record["url"], planet_url)])

    # Commit the relationship data after all inserts
    conn.commit()

def fetch_data_by_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch {url}: {response.status_code}")
        return []

# Fetch and store Star Wars data
def main():
    tables = {
        "people": ["name", "birth_year", "eye_color", "gender", "hair_color", "height", "mass", "skin_color", "homeworld", "url", "created", "edited", "films", "species", "starships", "vehicles"],
        "films": ["title", "episode_id", "opening_crawl", "director", "producer", "release_date", "url", "created", "edited", "species", "starships", "vehicles", "planets"],
        "planets": ["name", "diameter", "rotation_period", "orbital_period", "gravity", "population", "climate", "terrain", "surface_water", "url", "created", "edited"],
        "species": ["name", "classification", "designation", "average_height", "average_lifespan", "eye_colors", "hair_colors", "skin_colors", "language", "homeworld", "url", "created", "edited"],
        "vehicles": ["name", "model", "vehicle_class", "manufacturer", "length", "cost_in_credits", "crew", "passengers", "max_atmosphering_speed", "cargo_capacity", "consumables", "url", "created", "edited"],
        "starships": ["name", "model", "starship_class", "manufacturer", "cost_in_credits", "length", "crew", "passengers", "max_atmosphering_speed", "hyperdrive_rating", "MGLT", "cargo_capacity", "consumables", "url", "created", "edited"]
    }
    
    # Phase 1: Insert non-relationship data for all tables
    all_connections = []
    for table, columns in tables.items():
        print(f"\nFetching {table}...")
        data = fetch_data(table)
        if data:
            print(f"Inserting {len(data)} records into {table}...")
            conn, table, data, columns = insert_data(table, data, columns)
            all_connections.append((conn, table, data, columns))
        else:
            print(f"No data fetched for {table}.")

    # Phase 2: Insert relationship data for all tables
    for conn, table, data, columns in all_connections:
        print(f"Inserting relationship data for {table}...")
        insert_relationship_data(conn, table, data, columns)
        conn.close()  # Close each connection after handling it

if __name__ == "__main__":
    main()


Star Wars Schema executed successfully.

Fetching people...
Inserting 82 records into people...
Inserting non-relationship data into table: people

Fetching films...
Inserting 6 records into films...
Inserting non-relationship data into table: films

Fetching planets...
Inserting 60 records into planets...
Inserting non-relationship data into table: planets

Fetching species...
Inserting 37 records into species...
Inserting non-relationship data into table: species

Fetching vehicles...
Inserting 39 records into vehicles...
Inserting non-relationship data into table: vehicles

Fetching starships...
Inserting 36 records into starships...
Inserting non-relationship data into table: starships
Inserting relationship data for people...
Inserting into people_films: person_url=https://swapi.dev/api/people/1/ film_url=https://swapi.dev/api/films/1/
Inserting into people_films: person_url=https://swapi.dev/api/people/1/ film_url=https://swapi.dev/api/films/2/
Inserting into people_films: person

## Question 3

We first provide the DDL script used to create the tables required for the data from the OMDB API.

This file is conatined in the `ddl` folder. Running this SQL code is NOT required as the following Python code handles it.


In [ ]:
-- Table to store rating providers (e.g., Internet Movie Database, Rotten Tomatoes, Metacritic)
CREATE TABLE rating_providers (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE NOT NULL
);

-- Table to store movie ratings
CREATE TABLE ratings (
    id SERIAL PRIMARY KEY,
    film_id SERIAL NOT NULL,
    imdb_id TEXT NOT NULL,  -- IMDb ID for the movie
    rating_provider_id INT NOT NULL,
    rating_value TEXT NOT NULL,  -- Ratings may be in different formats (e.g., "8.4/10", "94%", "78/100")
    FOREIGN KEY (rating_provider_id) REFERENCES rating_providers(id) ON DELETE CASCADE,
    FOREIGN KEY (film_id) REFERENCES films(id) ON DELETE CASCADE
);

We continue with the Python script used to fetch data from the OMDB API.


In [34]:
import requests
import psycopg2
from dotenv import load_dotenv
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Database connection
conn = psycopg2.connect(
    dbname=os.getenv("UNNORMALIZED_DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)
cur = conn.cursor()

SCHEMA_FILE = "ddl/02_omdb_tables.sql"

try:
    # Read the SQL schema file
    with open(SCHEMA_FILE, "r", encoding="utf-8") as file:
        sql_script = file.read()

    # Execute the SQL script
    cur.execute(sql_script)

    print("OMDB Schema executed successfully.")

except Exception as e:
    print(f"Error executing schema: {e}")

# OMDB API Key
OMDB_BASE_URL = "http://www.omdbapi.com/"

# Fetch all films (title and release year)
print("Fetching films from database...")
cur.execute("SELECT id, title, release_date FROM films;")
films = cur.fetchall()
print(f"Retrieved {len(films)} films.")

# Function to get movie ratings from OMDB API
def fetch_movie_ratings(title, year):
    params = {
        "t": title,
        "y": year,
        "apikey": os.getenv("OMDB_API_KEY")
    }
    print(f"Requesting OMDB API for {title} ({year})...")
    response = requests.get(OMDB_BASE_URL, params=params)
    if response.status_code == 200:
        print(f"Response received for {title} ({year})")
        return response.json()
    else:
        print(f"Failed to fetch data for {title} ({year}) - Status Code: {response.status_code}")
        return None

# Insert rating providers
print("Fetching existing rating providers...")
cur.execute("SELECT id, name FROM rating_providers;")
existing_providers = {name: id for id, name in cur.fetchall()}
print(f"Retrieved {len(existing_providers)} rating providers.")

# Process each film
for film_id, title, release_date in films:
    year = release_date.year if release_date else None
    if not year:
        print(f"Skipping {title} due to missing release year.")
        continue

    print(f"\nFetching ratings for {title} ({year})")
    movie_data = fetch_movie_ratings(title, year)
    
    if not movie_data or movie_data.get("Response") == "False":
        print(f"No data found for {title} ({year}).")
        continue
    
    imdb_id = movie_data.get("imdbID", "N/A")
    print(f"IMDb ID for {title} ({year}): {imdb_id}")
    
    print(f"Processing ratings for {title} ({year})")
    for rating in movie_data.get("Ratings", []):
        provider_name = rating["Source"]
        rating_value = rating["Value"]
        print(f"Found rating: {provider_name} - {rating_value}")
        
        # Ensure provider exists
        if provider_name not in existing_providers:
            print(f"Inserting new rating provider: {provider_name}")
            cur.execute("INSERT INTO rating_providers (name) VALUES (%s) RETURNING id;", (provider_name,))
            provider_id = cur.fetchone()[0]
            existing_providers[provider_name] = provider_id
            conn.commit()
        else:
            provider_id = existing_providers[provider_name]
        
        # Insert rating with imdb_id
        print(f"Inserting rating for {title}: IMDb ID {imdb_id}, Provider ID {provider_id}, Value {rating_value}")
        cur.execute(
            """
            INSERT INTO ratings (film_id, imdb_id, rating_provider_id, rating_value)
            VALUES (%s, %s, %s, %s);
            """,
            (str(film_id), str(imdb_id), provider_id, rating_value)  # Ensuring correct types
        )
        conn.commit()

print("\nClosing database connection...")
cur.close() 
conn.close()
print("\nRatings data populated successfully.")


OMDB Schema executed successfully.
Fetching films from database...
Retrieved 6 films.
Fetching existing rating providers...
Retrieved 0 rating providers.

Fetching ratings for A New Hope (1977)
Requesting OMDB API for A New Hope (1977)...
Response received for A New Hope (1977)
IMDb ID for A New Hope (1977): tt0076759
Processing ratings for A New Hope (1977)
Found rating: Internet Movie Database - 8.6/10
Inserting new rating provider: Internet Movie Database
Inserting rating for A New Hope: IMDb ID tt0076759, Provider ID 1, Value 8.6/10
Found rating: Rotten Tomatoes - 93%
Inserting new rating provider: Rotten Tomatoes
Inserting rating for A New Hope: IMDb ID tt0076759, Provider ID 2, Value 93%
Found rating: Metacritic - 90/100
Inserting new rating provider: Metacritic
Inserting rating for A New Hope: IMDb ID tt0076759, Provider ID 3, Value 90/100

Fetching ratings for The Empire Strikes Back (1980)
Requesting OMDB API for The Empire Strikes Back (1980)...
Response received for The Empi

# Question 4

After downloading the TMDB Movie dataset from Kaggle, the following DDL script was used to create the tables necessary for the data to be collected from TMDB.

This file is conatined in the `ddl` folder. Running this SQL code is NOT required as the following Python code handles it.


In [ ]:
-- Delete the movie_metadata table if it exists
DROP TABLE IF EXISTS movie_metadata;

-- Table to store additional movie metadata (popularity and keywords)
CREATE TABLE movie_metadata (
    id INT PRIMARY KEY,
    imdb_id TEXT UNIQUE NOT NULL,
    popularity NUMERIC NOT NULL,
    keywords TEXT NOT NULL,
    FOREIGN KEY (id) REFERENCES films(id) ON DELETE CASCADE
);


We then use Python to fetch data from the `.csv` file downloaded and use the Star Wars movie data only.


In [35]:
import csv
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv("UNNORMALIZED_DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)
cur = conn.cursor()

SCHEMA_FILE = "ddl/03_tmdb_tables.sql"

try:
    # Read the SQL schema file
    with open(SCHEMA_FILE, "r", encoding="utf-8") as file:
        sql_script = file.read()

    # Execute the SQL script
    cur.execute(sql_script)

    print("TMDB Schema executed successfully.")

except Exception as e:
    print(f"Error executing schema: {e}")
          
# Path to the CSV file
current_directory = os.getcwd()
csv_file_path = os.path.join(current_directory, "data", "tmdb_filtered_dataset.csv")

# Read IMDb IDs from ratings table and map them to film ids
print("Fetching IMDb IDs and corresponding film ids from ratings table...")
cur.execute("""
    SELECT r.imdb_id, f.id 
    FROM ratings r
    JOIN films f ON r.film_id = f.id;
""")
imdb_to_film_id = {row[0]: row[1] for row in cur.fetchall()}
print(f"Retrieved {len(imdb_to_film_id)} IMDb IDs and film IDs.")

# Read CSV and insert relevant data
with open(csv_file_path, mode="r", encoding="utf-8") as csv_file:
    reader = csv.DictReader(csv_file)

    for row in reader:
        imdb_id = row.get("imdb_id")
        popularity = row.get("popularity")
        keywords = row.get("keywords")

        # Skip rows with missing data or if imdb_id is not found in ratings table
        if not imdb_id or imdb_id not in imdb_to_film_id or not popularity or not keywords:
            continue

        film_id = imdb_to_film_id[imdb_id]

        print(f"Inserting film_id {film_id} (IMDb ID: {imdb_id}) with popularity {popularity} and keywords {keywords}")

        # Insert into the movie_metadata table, ensuring the film_id is used as a reference
        cur.execute(
            """
            INSERT INTO movie_metadata (id, imdb_id, popularity, keywords)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (id) DO NOTHING;
            """,
            (film_id, imdb_id, popularity, keywords)
        )
        conn.commit()

print("Closing database connection...")
cur.close()
conn.close()
print("Movie metadata populated successfully.")

TMDB Schema executed successfully.
Fetching IMDb IDs and corresponding film ids from ratings table...
Retrieved 6 IMDb IDs and film IDs.
Inserting film_id 1 (IMDb ID: tt0076759) with popularity 88.559 and keywords empire, galaxy, rebellion, android, hermit, smuggling (contraband), freedom, rebel, rescue mission, space, planet, desert, super power, oppression, space opera, wizard, totalitarianism
Inserting film_id 2 (IMDb ID: tt0080684) with popularity 35.265 and keywords rebellion, android, spacecraft, asteroid, rebel, space battle, snowstorm, space colony, swamp, space opera, artic
Inserting film_id 3 (IMDb ID: tt0086190) with popularity 40.104 and keywords spacecraft, sibling relationship, rebel, emperor, space battle, matter of life and death, forest, desert, space opera
Inserting film_id 4 (IMDb ID: tt0120915) with popularity 46.845 and keywords galaxy, senate, taskmaster, prophecy, queen, apprentice, taxes, space opera
Inserting film_id 6 (IMDb ID: tt0121766) with popularity 41.21

## Question 5

After retrieving the initial set of data, and investigating the overall structure of the unnormalized database, we will be using the following DDL script to structure the normalized dataset.

This file is conatined in the `ddl` folder. Running this SQL code is NOT required as the following Python code handles it.


In [ ]:
-- Drop tables if they exist
DROP TABLE IF EXISTS people CASCADE;
DROP TABLE IF EXISTS films CASCADE;
DROP TABLE IF EXISTS starships CASCADE;
DROP TABLE IF EXISTS vehicles CASCADE;
DROP TABLE IF EXISTS species CASCADE;
DROP TABLE IF EXISTS planets CASCADE;

DROP TABLE IF EXISTS people_films CASCADE;
DROP TABLE IF EXISTS people_species CASCADE;
DROP TABLE IF EXISTS people_starships CASCADE;
DROP TABLE IF EXISTS people_vehicles CASCADE;
DROP TABLE IF EXISTS films_species CASCADE;
DROP TABLE IF EXISTS films_starships CASCADE;
DROP TABLE IF EXISTS films_vehicles CASCADE;
DROP TABLE IF EXISTS films_planets CASCADE;

DROP TABLE IF EXISTS rating_providers CASCADE;
DROP TABLE IF EXISTS ratings CASCADE;

DROP TABLE IF EXISTS keywords CASCADE;
DROP TABLE IF EXISTS movie_metadata CASCADE;

-- Create tables
CREATE TABLE people (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    birth_year VARCHAR(20),
    eye_color VARCHAR(50),
    gender VARCHAR(20),
    hair_color VARCHAR(50),
    height VARCHAR(20),
    mass VARCHAR(20),
    skin_color VARCHAR(50),
    homeworld VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE films (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    episode_id INTEGER NOT NULL,
    opening_crawl TEXT,
    director VARCHAR(255),
    producer VARCHAR(255),
    release_date DATE,
    imdb_id VARCHAR(50),  -- Added IMDb ID
    popularity NUMERIC,
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE starships (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    model VARCHAR(255),
    starship_class VARCHAR(255),
    manufacturer VARCHAR(255),
    cost_in_credits VARCHAR(50),
    length VARCHAR(50),
    crew VARCHAR(50),
    passengers VARCHAR(50),
    max_atmosphering_speed VARCHAR(50),
    hyperdrive_rating VARCHAR(50),
    MGLT VARCHAR(50),
    cargo_capacity VARCHAR(50),
    consumables VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE vehicles (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    model VARCHAR(255),
    vehicle_class VARCHAR(255),
    manufacturer VARCHAR(255),
    length VARCHAR(50),
    cost_in_credits VARCHAR(50),
    crew VARCHAR(50),
    passengers VARCHAR(50),
    max_atmosphering_speed VARCHAR(50),
    cargo_capacity VARCHAR(50),
    consumables VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE species (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    classification VARCHAR(255),
    designation VARCHAR(50),
    average_height VARCHAR(50),
    average_lifespan VARCHAR(50),
    eye_colors VARCHAR(255),
    hair_colors VARCHAR(255),
    skin_colors VARCHAR(255),
    language VARCHAR(255),
    homeworld VARCHAR(255),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

CREATE TABLE planets (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    diameter VARCHAR(50),
    rotation_period VARCHAR(50),
    orbital_period VARCHAR(50),
    gravity VARCHAR(50),
    population VARCHAR(50),
    climate VARCHAR(255),
    terrain VARCHAR(255),
    surface_water VARCHAR(50),
    url VARCHAR(255) UNIQUE NOT NULL,
    created TIMESTAMP,
    edited TIMESTAMP
);

-- Relationship Tables for Many-to-Many Relationships with IDs
CREATE TABLE people_films (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, film_url)
);

CREATE TABLE people_species (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    species_url VARCHAR(255) REFERENCES species(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, species_url)
);

CREATE TABLE people_starships (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    starship_url VARCHAR(255) REFERENCES starships(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, starship_url)
);

CREATE TABLE people_vehicles (
    person_url VARCHAR(255) REFERENCES people(url) ON DELETE CASCADE,
    vehicle_url VARCHAR(255) REFERENCES vehicles(url) ON DELETE CASCADE,
    PRIMARY KEY (person_url, vehicle_url)
);

CREATE TABLE films_species (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    species_url VARCHAR(255) REFERENCES species(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, species_url)
);

CREATE TABLE films_starships (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    starship_url VARCHAR(255) REFERENCES starships(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, starship_url)
);

CREATE TABLE films_vehicles (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    vehicle_url VARCHAR(255) REFERENCES vehicles(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, vehicle_url)
);

CREATE TABLE films_planets (
    film_url VARCHAR(255) REFERENCES films(url) ON DELETE CASCADE,
    planet_url TEXT REFERENCES planets(url) ON DELETE CASCADE,
    PRIMARY KEY (film_url, planet_url)
);

-- Table to store rating providers (e.g., Internet Movie Database, Rotten Tomatoes, Metacritic)
CREATE TABLE rating_providers (
    id SERIAL PRIMARY KEY,
    name TEXT UNIQUE NOT NULL
);

-- Table to store movie ratings
CREATE TABLE ratings (
    id SERIAL PRIMARY KEY,
    film_id INT NOT NULL,
    rating_provider_id INT NOT NULL,
    rating_value NUMERIC NOT NULL,  -- Rating value is now numeric (e.g., 8.4, 94, 78)
    FOREIGN KEY (rating_provider_id) REFERENCES rating_providers(id) ON DELETE CASCADE,
    FOREIGN KEY (film_id) REFERENCES films(id) ON DELETE CASCADE
);

-- Table to store keywords
CREATE TABLE keywords (
    id SERIAL PRIMARY KEY,
    keyword TEXT UNIQUE NOT NULL
);

-- Table to store additional movie metadata
CREATE TABLE movie_metadata (
    id SERIAL PRIMARY KEY,
    film_id INT NOT NULL,
    keyword_id INT NOT NULL,
    FOREIGN KEY (film_id) REFERENCES films(id) ON DELETE CASCADE,
    FOREIGN KEY (keyword_id) REFERENCES keywords(id) ON DELETE CASCADE
);


## Question 6

We make use of batch queries and inserts to take data from our unnormalized database and place it into the normalized database, through Python.


In [36]:
import psycopg2

from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

starwars_db_conn = psycopg2.connect(
    dbname=os.getenv("UNNORMALIZED_DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

normalized_db_conn = psycopg2.connect(
    dbname=os.getenv("NORMALIZED_DB_NAME"),
    user= os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

# Create a cursor for both databases
starwars_db_cursor = starwars_db_conn.cursor()
normalized_db_cursor = normalized_db_conn.cursor()

SCHEMA_FILE = "ddl/04_final_schema.sql"

try:
    # Read the SQL schema file
    with open(SCHEMA_FILE, "r", encoding="utf-8") as file:
        sql_script = file.read()

    # Execute the SQL script
    normalized_db_cursor.execute(sql_script)

    print("TMDB Schema executed successfully.")

except Exception as e:
    print(f"Error executing schema: {e}")

# ============================================================
# PHASE 1: Migrating 'films' Data from starwars_db to normalized_starwars_db
# ============================================================

# Query to get data from the 'films' table and the 'popularity' from 'movie_metadata' in starwars_db
starwars_db_cursor.execute("""
    SELECT 
        f.id, f.title, f.episode_id, f.opening_crawl, f.director, f.producer, 
        f.release_date, f.url, f.created, f.edited, m.popularity, m.imdb_id
    FROM films f
    LEFT JOIN movie_metadata m ON f.id = m.id
""")

# Fetch all rows from the query result
films = starwars_db_cursor.fetchall()

# Insert data into the 'films' table in normalized_starwars_db
for film in films:
    id, title, episode_id, opening_crawl, director, producer, release_date, url, created, edited, popularity, imdb_id = film
    
    # Prepare the SQL statement to insert into the normalized films table
    insert_query = """
        INSERT INTO films (id, title, episode_id, opening_crawl, director, producer, 
                           release_date, url, created, edited, popularity, imdb_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Execute the insertion in the normalized_starwars_db
    normalized_db_cursor.execute(insert_query, (id, title, episode_id, opening_crawl, director, producer, 
                                                release_date, url, created, edited, popularity, imdb_id))

# Commit the transaction and close the connections
normalized_db_conn.commit()

# ============================================================
# End of PHASE 1
# ============================================================

# ============================================================
# PHASE 2: Migrating 'people', 'starships', 'vehicles', 'species', and 'planets' Data from starwars_db to normalized_starwars_db
# ============================================================

# People Table
starwars_db_cursor.execute("SELECT id, name, birth_year, eye_color, gender, hair_color, height, mass, skin_color, homeworld, url, created, edited FROM people")
people = starwars_db_cursor.fetchall()
for person in people:
    id, name, birth_year, eye_color, gender, hair_color, height, mass, skin_color, homeworld, url, created, edited = person
    insert_query = """
        INSERT INTO people (id, name, birth_year, eye_color, gender, hair_color, height, mass, skin_color, homeworld, url, created, edited)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    normalized_db_cursor.execute(insert_query, (id, name, birth_year, eye_color, gender, hair_color, height, mass, skin_color, homeworld, url, created, edited))

# Starships Table
starwars_db_cursor.execute("SELECT id, name, model, starship_class, manufacturer, cost_in_credits, length, crew, passengers, max_atmosphering_speed, hyperdrive_rating, MGLT, cargo_capacity, consumables, url, created, edited FROM starships")
starships = starwars_db_cursor.fetchall()
for starship in starships:
    # Unpack the data into corresponding variables
    id, name, model, starship_class, manufacturer, cost_in_credits, length, crew, passengers, max_atmosphering_speed, hyperdrive_rating, MGLT, cargo_capacity, consumables, url, created, edited = starship
    
    # Prepare the SQL statement to insert into the normalized starships table
    insert_query = """
        INSERT INTO starships (id, name, model, starship_class, manufacturer, cost_in_credits, length, crew, passengers, max_atmosphering_speed, hyperdrive_rating, MGLT, cargo_capacity, consumables, url, created, edited)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Execute the insertion in the normalized_starwars_db
    normalized_db_cursor.execute(insert_query, (id, name, model, starship_class, manufacturer, cost_in_credits, length, crew, passengers, max_atmosphering_speed, hyperdrive_rating, MGLT, cargo_capacity, consumables, url, created, edited))

# Vehicles Table
starwars_db_cursor.execute("SELECT id, name, model, vehicle_class, manufacturer, length, cost_in_credits, crew, passengers, max_atmosphering_speed, cargo_capacity, consumables, url, created, edited FROM vehicles")
vehicles = starwars_db_cursor.fetchall()
for vehicle in vehicles:
    id, name, model, vehicle_class, manufacturer, length, cost_in_credits, crew, passengers, max_atmosphering_speed, cargo_capacity, consumables, url, created, edited = vehicle
    insert_query = """
        INSERT INTO vehicles (id, name, model, vehicle_class, manufacturer, length, cost_in_credits, crew, passengers, max_atmosphering_speed, cargo_capacity, consumables, url, created, edited)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    normalized_db_cursor.execute(insert_query, (id, name, model, vehicle_class, manufacturer, length, cost_in_credits, crew, passengers, max_atmosphering_speed, cargo_capacity, consumables, url, created, edited))

# Species Table
starwars_db_cursor.execute("SELECT id, name, classification, designation, average_height, average_lifespan, eye_colors, hair_colors, skin_colors, language, homeworld, url, created, edited FROM species")
species = starwars_db_cursor.fetchall()
for spec in species:
    id, name, classification, designation, average_height, average_lifespan, eye_colors, hair_colors, skin_colors, language, homeworld, url, created, edited = spec
    insert_query = """
        INSERT INTO species (id, name, classification, designation, average_height, average_lifespan, eye_colors, hair_colors, skin_colors, language, homeworld, url, created, edited)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    normalized_db_cursor.execute(insert_query, (id, name, classification, designation, average_height, average_lifespan, eye_colors, hair_colors, skin_colors, language, homeworld, url, created, edited))

# Planets Table
starwars_db_cursor.execute("SELECT id, name, diameter, rotation_period, orbital_period, gravity, population, climate, terrain, surface_water, url, created, edited FROM planets")
planets = starwars_db_cursor.fetchall()
for planet in planets:
    id, name, diameter, rotation_period, orbital_period, gravity, population, climate, terrain, surface_water, url, created, edited = planet
    insert_query = """
        INSERT INTO planets (id, name, diameter, rotation_period, orbital_period, gravity, population, climate, terrain, surface_water, url, created, edited)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    normalized_db_cursor.execute(insert_query, (id, name, diameter, rotation_period, orbital_period, gravity, population, climate, terrain, surface_water, url, created, edited))

# Commit the transaction
normalized_db_conn.commit()

# ============================================================
# End of PHASE 2
# ============================================================

# ============================================================
# PHASE 3: Migrating Relationships Data and Extracting IDs from URLs for Relationship Tables
# ============================================================

# People_Films Relationship
starwars_db_cursor.execute("SELECT person_url, film_url FROM people_films")
people_films = starwars_db_cursor.fetchall()
for person_url, film_url in people_films:
    
    insert_query = """
        INSERT INTO people_films (person_url, film_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (person_url, film_url))

# People_Species Relationship
starwars_db_cursor.execute("SELECT person_url, species_url FROM people_species")
people_species = starwars_db_cursor.fetchall()
for person_url, species_url in people_species:
    
    insert_query = """
        INSERT INTO people_species (person_url, species_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (person_url, species_url))

# People_Starships Relationship
starwars_db_cursor.execute("SELECT person_url, starship_url FROM people_starships")
people_starships = starwars_db_cursor.fetchall()
for person_url, starship_url in people_starships:
    
    insert_query = """
        INSERT INTO people_starships (person_url, starship_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (person_url, starship_url))

# People_Vehicles Relationship
starwars_db_cursor.execute("SELECT person_url, vehicle_url FROM people_vehicles")
people_vehicles = starwars_db_cursor.fetchall()
for person_url, vehicle_url in people_vehicles:
    
    insert_query = """
        INSERT INTO people_vehicles (person_url, vehicle_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (person_url, vehicle_url))

# Films_Species Relationship
starwars_db_cursor.execute("SELECT film_url, species_url FROM films_species")
films_species = starwars_db_cursor.fetchall()
for film_url, species_url in films_species:
    
    insert_query = """
        INSERT INTO films_species (film_url, species_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (film_url, species_url))

# Films_Starships Relationship
starwars_db_cursor.execute("SELECT film_url, starship_url FROM films_starships")
films_starships = starwars_db_cursor.fetchall()
for film_url, starship_url in films_starships:
    
    insert_query = """
        INSERT INTO films_starships (film_url, starship_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (film_url, starship_url))

# Films_Vehicles Relationship
starwars_db_cursor.execute("SELECT film_url, vehicle_url FROM films_vehicles")
films_vehicles = starwars_db_cursor.fetchall()
for film_url, vehicle_url in films_vehicles:
    
    insert_query = """
        INSERT INTO films_vehicles (film_url, vehicle_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (film_url, vehicle_url))

# Films_Planets Relationship
starwars_db_cursor.execute("SELECT film_url, planet_url FROM films_planets")
films_planets = starwars_db_cursor.fetchall()
for film_url, planet_url in films_planets:
    
    insert_query = """
        INSERT INTO films_planets (film_url, planet_url)
        VALUES (%s, %s)
    """
    normalized_db_cursor.execute(insert_query, (film_url, planet_url))

# Commit the transaction
normalized_db_conn.commit()

# ============================================================
# End of PHASE 3
# ============================================================

# ============================================================
# PHASE 4: Migrating Ratings and Rating Providers Data from starwars_db to normalized_starwars_db
# ============================================================

# Rating Providers Table
# First, we extract distinct rating providers from the 'rating_providers' table in the starwars_db
starwars_db_cursor.execute("SELECT DISTINCT name FROM rating_providers WHERE name IS NOT NULL")
rating_providers = starwars_db_cursor.fetchall()

# Insert each rating provider into the normalized_rating_providers table
for provider in rating_providers:
    provider_name = provider[0]
    insert_query = """
        INSERT INTO rating_providers (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING
    """
    normalized_db_cursor.execute(insert_query, (provider_name,))

# Commit the transaction to save the rating providers
normalized_db_conn.commit()

# Ratings Table
# Now, we migrate the ratings from starwars_db to normalized_starwars_db
# Note: The IMDb ID will be excluded

# Fetch film ratings from movie_metadata (excluding IMDb ID)
starwars_db_cursor.execute("""
    SELECT film_id, rating_provider_id, rating_value
    FROM ratings
""")

ratings = starwars_db_cursor.fetchall()

# Insert the ratings into the normalized ratings table
for rating in ratings:
    film_id, rating_provider_id, rating_value = rating
    
    # Convert the rating_value to a decimal score (e.g., 8.4 becomes 84, 94% becomes 94, 0.92 becomes 92)
    if '%' in rating_value:
        rating_value = float(rating_value.strip('%'))  # Convert to numeric (e.g., "94%" becomes 94)
    elif '/' in rating_value:
        # Extract the numeric part before the '/' and the denominator after the '/'
        numerator, denominator = rating_value.split('/')
        rating_value = float(numerator)  # Convert "8.6/10" to 8.6 or "9/100" to 9
        denominator = float(denominator)  # Get the denominator (10 or 100)
        if denominator == 10:
            rating_value *= 10  # Convert "8.6/10" to 86
        elif denominator == 100:
            rating_value *= 1  # Leave as is for "/100" case (e.g., "9/100" becomes 9)
    elif '.' in rating_value:  # If the rating is a decimal like "0.92"
        rating_value = float(rating_value)  # Convert to numeric (e.g., 0.92 becomes 0.92)
        if rating_value < 1:  # Only multiply by 100 if it's a decimal
            rating_value *= 100  # Convert to percentage (e.g., 0.92 becomes 92)
        else:
            rating_value *= 10  # For values like 8.4, multiply by 10 to get 84
    else:
        rating_value = float(rating_value)  # If it's a whole number (e.g., 8, 9)

    # Ensure the final value is in a reasonable range (between 0 and 100)
    rating_value = min(max(rating_value, 0), 100)

    # Prepare the SQL statement to insert into the ratings table
    insert_query = """
        INSERT INTO ratings (film_id, rating_provider_id, rating_value)
        VALUES (%s, %s, %s)
    """
    
    # Execute the insertion in the normalized_starwars_db
    normalized_db_cursor.execute(insert_query, (film_id, rating_provider_id, rating_value))

# Commit the transaction and close the connections
normalized_db_conn.commit()

# ============================================================
# End of PHASE 4
# ============================================================

# ============================================================
# PHASE 5: Migrating Movie Metadata to Normalized Structure
# ============================================================

# Fetch all movie metadata from the old movie_metadata table
starwars_db_cursor.execute("""
    SELECT id, popularity, keywords
    FROM movie_metadata
""")

movie_metadata_rows = starwars_db_cursor.fetchall()

# Insert keywords into the normalized `keywords` table
for metadata in movie_metadata_rows:
    film_id, popularity, keywords = metadata

    # Split the keywords into a list (assuming they are comma-separated)
    keyword_list = keywords.split(',') if keywords else []
    
    for keyword in keyword_list:
        # Strip whitespace and handle duplicates
        keyword = keyword.strip()
        
        if keyword:
            # Check if the keyword already exists in the `keywords` table
            normalized_db_cursor.execute("SELECT id FROM keywords WHERE keyword = %s", (keyword,))
            keyword_row = normalized_db_cursor.fetchone()
            
            if keyword_row:
                keyword_id = keyword_row[0]
            else:
                # Insert new keyword if it doesn't exist
                normalized_db_cursor.execute("INSERT INTO keywords (keyword) VALUES (%s) RETURNING id", (keyword,))
                keyword_id = normalized_db_cursor.fetchone()[0]

            # Insert into the new movie_metadata table
            normalized_db_cursor.execute("""
                INSERT INTO movie_metadata (film_id, keyword_id)
                VALUES (%s, %s)
            """, (film_id, keyword_id))

# Commit the changes to the normalized database
normalized_db_conn.commit()

# ============================================================
# End of PHASE 5
# ============================================================

# Close the cursors and connections
starwars_db_cursor.close()
normalized_db_cursor.close()
starwars_db_conn.close()
normalized_db_conn.close()

print("Films table populated successfully in normalized_starwars_db!")


TMDB Schema executed successfully.
Films table populated successfully in normalized_starwars_db!


## Question 7

The following queries can be used to obtain the requested information:


### 7.1

Find the total number of movies, total number of planets, total number of people in the database.


In [ ]:
SELECT 
    (SELECT COUNT(*) FROM films) AS total_movies,
    (SELECT COUNT(*) FROM planets) AS total_planets,
    (SELECT COUNT(*) FROM people) AS total_people;

In [37]:
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

DB_PARAMS = {
    "dbname": os.getenv("NORMALIZED_DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT")
}

query = """
SELECT 
    (SELECT COUNT(*) FROM films) AS total_movies,
    (SELECT COUNT(*) FROM planets) AS total_planets,
    (SELECT COUNT(*) FROM people) AS total_people;
"""

try:
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchone()
            print(f"Total Movies: {result[0]}, Total Planets: {result[1]}, Total People: {result[2]}")
except Exception as e:
    print(f"Error executing query: {e}")

Total Movies: 6, Total Planets: 60, Total People: 82


### 7.2

Find top 3 movies with highest number of keywords.


In [ ]:
SELECT 
    f.title,
    COUNT(km.keyword_id) AS num_keywords
FROM 
    films f
JOIN 
    movie_metadata km ON f.id = km.film_id
GROUP BY 
    f.id
ORDER BY 
    num_keywords DESC
LIMIT 3;

In [38]:
query = """
SELECT 
    f.title,
    COUNT(km.keyword_id) AS num_keywords
FROM 
    films f
JOIN 
    movie_metadata km ON f.id = km.film_id
GROUP BY 
    f.id
ORDER BY 
    num_keywords DESC
LIMIT 3;
"""

try:
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            results = cursor.fetchall()
            print("Top 3 Movies with Most Keywords:")
            for row in results:
                print(f"Movie: {row[0]}, Keywords: {row[1]}")
except Exception as e:
    print(f"Error executing query: {e}")


Top 3 Movies with Most Keywords:
Movie: A New Hope, Keywords: 17
Movie: Attack of the Clones, Keywords: 13
Movie: The Empire Strikes Back, Keywords: 11


### 7.3

Find popular keyword(s) and movies associated with them.


In [ ]:
SELECT 
    k.keyword,
    COUNT(km.film_id) AS num_associations
FROM 
    keywords k
JOIN 
    movie_metadata km ON k.id = km.keyword_id
GROUP BY 
    k.id
ORDER BY 
    num_associations DESC;

In [39]:
query = """
SELECT 
    k.keyword,
    COUNT(km.film_id) AS num_associations
FROM 
    keywords k
JOIN 
    movie_metadata km ON k.id = km.keyword_id
GROUP BY 
    k.id
ORDER BY 
    num_associations DESC;
"""

try:
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            results = cursor.fetchall()
            print("Popular Keywords and Movie Associations:")
            for row in results:
                print(f"Keyword: {row[0]}, Associations: {row[1]}")
except Exception as e:
    print(f"Error executing query: {e}")


Popular Keywords and Movie Associations:
Keyword: space opera, Associations: 6
Keyword: rebel, Associations: 3
Keyword: spacecraft, Associations: 2
Keyword: galaxy, Associations: 2
Keyword: desert, Associations: 2
Keyword: android, Associations: 2
Keyword: space battle, Associations: 2
Keyword: senate, Associations: 2
Keyword: rebellion, Associations: 2
Keyword: cult figure, Associations: 2
Keyword: freedom, Associations: 1
Keyword: showdown, Associations: 1
Keyword: wedding, Associations: 1
Keyword: hatred, Associations: 1
Keyword: smuggling (contraband), Associations: 1
Keyword: teenage rebellion, Associations: 1
Keyword: emperor, Associations: 1
Keyword: dream sequence, Associations: 1
Keyword: artic, Associations: 1
Keyword: asteroid, Associations: 1
Keyword: sibling relationship, Associations: 1
Keyword: prophecy, Associations: 1
Keyword: taskmaster, Associations: 1
Keyword: alien race, Associations: 1
Keyword: snowstorm, Associations: 1
Keyword: good becoming evil, Associations: 

### 7.4

Find top ranked movies for each rating provider.


In [ ]:
SELECT 
    rp.name AS rating_provider,
    f.title,
    r.rating_value
FROM 
    ratings r
JOIN 
    films f ON r.film_id = f.id
JOIN 
    rating_providers rp ON r.rating_provider_id = rp.id
WHERE 
    (r.rating_provider_id, r.rating_value) IN (
        SELECT 
            r.rating_provider_id, MAX(r.rating_value)
        FROM 
            ratings r
        GROUP BY 
            r.rating_provider_id
    )
ORDER BY 
    rp.name;

In [40]:
query = """
SELECT 
    rp.name AS rating_provider,
    f.title,
    r.rating_value
FROM 
    ratings r
JOIN 
    films f ON r.film_id = f.id
JOIN 
    rating_providers rp ON r.rating_provider_id = rp.id
WHERE 
    (r.rating_provider_id, r.rating_value) IN (
        SELECT 
            r.rating_provider_id, MAX(r.rating_value)
        FROM 
            ratings r
        GROUP BY 
            r.rating_provider_id
    )
ORDER BY 
    rp.name;
"""

try:
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            results = cursor.fetchall()
            print("Top Ranked Movies per Rating Provider:")
            for row in results:
                print(f"Provider: {row[0]}, Movie: {row[1]}, Rating: {row[2]}")
except Exception as e:
    print(f"Error executing query: {e}")


Top Ranked Movies per Rating Provider:
Provider: Internet Movie Database, Movie: The Empire Strikes Back, Rating: 95.0
Provider: Metacritic, Movie: A New Hope, Rating: 90.0
Provider: Rotten Tomatoes, Movie: The Empire Strikes Back, Rating: 87.0


### 7.5

Write a batch-update query that rounds up all the ratings.


In [ ]:
UPDATE ratings
SET rating_value = CEIL(rating_value);

In [ ]:
query = """
UPDATE ratings
SET rating_value = CEIL(rating_value);
"""

try:
    with psycopg2.connect(**DB_PARAMS) as conn:
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print("Ratings rounded up successfully.")
except Exception as e:
    print(f"Error executing query: {e}")

# As indicated by the assignment, this query will not be run and is up for the grader to run.

## Question 8

The following two images depict the ERDs for the unnormalized and normalized databases.

These images were generated using pgAdmin 4.


### Unnormalized Database

![Test](erd/unnormalized_starwars_db.png)


### Normalized Database

![Test](erd/normalized_starwars_db.png)


## Question 9

To create the DML scripts for both unnormalized and normalized databases, the 'Backup' feature of pgAdmin 4 was used.

> Please refer to the `dml` folder to view them and populate the database.
